# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования могут быть учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открываем файл с данными и изучаем общую информацию

**Импорт библиотек**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#pip install pymystem3

In [2]:
import pandas as pd

# получаем стеммер/лемматизатор для слов на русском
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


откроем файл

In [3]:

try:
  data=pd.read_csv('datasets/data.csv')
except:
   data=pd.read_csv('/content/drive/MyDrive/Coursera и др/Анилитик данных/praktikum.yandex/Colab Notebooks/datasets/data.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6946,0,-911.218778,49,среднее,1,Не женат / не замужем,4,F,сотрудник,0,137899.012352,строительство жилой недвижимости
18256,0,-605.990492,37,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,128985.202781,покупка жилья
21382,1,-4560.953044,43,среднее,1,женат / замужем,0,F,сотрудник,0,208865.504736,автомобили
8904,0,396697.138575,54,среднее,1,женат / замужем,0,F,пенсионер,0,110839.364338,ремонт жилью
13210,0,-112.903414,22,высшее,0,женат / замужем,0,F,компаньон,0,230882.968662,покупка недвижимости


**Вывод**

В разных столбцах (days_employed, total_income) разное количество элементов с определёнными значениями. Следовательно, есть пропущенные значения. Данные нужно обработать.

## Шаг 2. Предобработка данных

### Обработка пропусков

ознакомимся с уникальными значениями по каждому столбцу

In [ ]:
for col in data.columns:
    print(col,data[col].unique())
    print('')

children [ 1  0  3  2 -1  4 20  5]

days_employed [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]

dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

education_id [0 1 2 3 4]

family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

family_status_id [0 1 2 3 4]

gender ['F' 'M' 'XNA']

income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

debt [0 1]

total_income [253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]

p

выведем кол-во пропущенных значений вида nan/none

In [ ]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


выведем процент пропущенных значений вида nan/none

In [ ]:
pd.DataFrame(round((data.isna().mean()*100),2)).style.background_gradient('coolwarm')

,0
children,0.000000
days_employed,10.100000
dob_years,0.000000
education,0.000000
education_id,0.000000
family_status,0.000000
family_status_id,0.000000
gender,0.000000
income_type,0.000000
debt,0.000000


__обработка children__

кол-во уникальных значений

In [ ]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 заменяем на 2 (предположим, что опечатка при вводе)

In [ ]:
data.loc[data['children'] == 20, 'children'] = 2

-1 заменяем на 1 (опечатка при вводе)

In [ ]:
data.loc[data['children'] == -1, 'children'] = 1
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

 Еще их можно просто отбросить: мы не знаем, какие значения там должны быть в действительности, и этих строк пренебрежимо мало в общем количестве.

__обработка days_employed__

Проанализируем данные

In [ ]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


избавляемся от отрицательных значений (возможно это опечатка при вводе)

In [ ]:
data['days_employed']=data['days_employed'].abs()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Изучаем пропуски

In [ ]:
data[data['days_employed'].isna()].sort_values(by='dob_years').tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1303,1,NaN,70,начальное,3,гражданский брак,1,F,сотрудник,0,NaN,операции с коммерческой недвижимостью
13994,0,NaN,70,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,приобретение автомобиля
7938,0,NaN,71,СРЕДНЕЕ,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
9604,0,NaN,71,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
6537,0,NaN,71,Среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
8456,0,NaN,71,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,операции с коммерческой недвижимостью
11548,0,NaN,71,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,покупка жилой недвижимости
13864,0,NaN,72,среднее,1,женат / замужем,0,F,компаньон,0,NaN,сделка с автомобилем
10563,0,NaN,72,Среднее,1,Не женат / не замужем,4,F,пенсионер,1,NaN,приобретение автомобиля
18664,1,NaN,73,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,дополнительное образование


* Заметим, что строки 7938, 9604, 6537 являются дублями и будут обработаны далее

значения столбца days_employed общий трудовой стаж, это количественная переменная. Пропуски в таких переменных заполняют характерными значениями.

получим медиану проработанных лет (360 - кол-во дней в году для стажа, для наглядности)

In [ ]:
data['days_employed'].median()/360

6.095057130218597

Среднее

In [ ]:
data['days_employed'].mean()/360

185.8742469633943

Среднее некоректно для трудового стажа. Предположим что часть значений сильно выделяются среди большинства.

Такой большой стаж у значений, которые не были отрицательными. А еще он характерен только для некоторых типов занятости — можно сгруппировать по income_type и посмотреть на медиану стажа по этим группам.

Убираем пропуски (заполняем медианой employ_median) и проверяем результат

In [ ]:
print('befor:',len(data[data['days_employed'].isna()]))
employ_median = data['days_employed'].median()
data['days_employed'].fillna(employ_median,inplace=True)
print('after:',data['days_employed'].isna().sum())

befor: 2174
after: 0


получим максимум проработанных лет (360 - кол-во дней в году для стажа)

In [ ]:
data['days_employed'].max()/360

1115.9872235425833

Посмотрим выделяющиеся данные

In [ ]:
data.sort_values(by='days_employed')['days_employed'].tail(5)

7794     401663.850046
2156     401674.466633
7664     401675.093434
10006    401715.811749
6954     401755.400475
Name: days_employed, dtype: float64

* посмотрим на них с головы. 
* возьмем 36000 дней, максимальное кол-во стажа за 100 лет
* посчитаем кол-во

In [ ]:
data[data['days_employed']>36000].sort_values(by='days_employed')['days_employed'].head()

20444    328728.720605
9328     328734.923996
17782    328771.341387
14783    328795.726728
7229     328827.345667
Name: days_employed, dtype: float64

In [ ]:
data[data['days_employed']>36000]['days_employed'].count()

3445

диапазон "неадекватных" данных от 328728.720605 до 401755.400475

значения столбца days_employed, это количественная переменная. Заполняем характерными значениями (медианой)

In [ ]:
employ_median = data['days_employed'].median()

In [ ]:
data.loc[data['days_employed']>328728,'days_employed']=employ_median

In [ ]:
data[data['days_employed']>328728]['days_employed'].count()

0

__обработка dob_years__

изучим данные в столбце

In [ ]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

надо обработать значение 0. 

возможная причина возникновения , отсутствие информации о возрасте при заполнении данных на клиента

In [ ]:
data[data['dob_years'] == 0]['dob_years'].count()

101

кол-во таких строк 0.4%

In [ ]:
101/21525

0.004692218350754936

взглянем на данные. отсортировав по доходу total_income

In [ ]:
data[data['dob_years'] == 0].sort_values(by='total_income').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14659,0,2194.220567,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,34974.450366,жилье
15295,0,1245.658244,0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,49082.259548,автомобиль
15886,0,2194.220567,0,среднее,1,женат / замужем,0,F,пенсионер,0,50230.794174,высшее образование
12729,0,2194.220567,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,54815.744162,образование
8061,0,2194.220567,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,61804.271999,высшее образование


вычислим среднее значение

In [ ]:
data['dob_years'].mean()

43.29337979094077

значения столбца dob_years, это количественная переменная. Заполним средним. и проверим результат

In [ ]:
dob_years_avg = int(data['dob_years'].mean())
print('средний возраст:',dob_years_avg)
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_avg
print('after:',data[data['dob_years'] == 0]['dob_years'].count())

средний возраст: 43
after: 0


__обработка gender__

In [ ]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

предположим что это опечатка

In [ ]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


замена XNA на F, как наиболее часто встречающийся

In [ ]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
data['gender'].value_counts()
#data.loc[data['gender'] == 'XNA']

F    14237
M     7288
Name: gender, dtype: int64

__обработка total_income ежемесячный доход__

Проанализируем данные

In [ ]:
data[data['total_income'].isna()].sort_values(by='dob_years').tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11710,0,2194.220567,70,ВЫСШЕЕ,0,женат / замужем,0,F,пенсионер,0,NaN,операции с коммерческой недвижимостью
13994,0,2194.220567,70,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,приобретение автомобиля
9604,0,2194.220567,71,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
6537,0,2194.220567,71,Среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
8456,0,2194.220567,71,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,операции с коммерческой недвижимостью
11548,0,2194.220567,71,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,покупка жилой недвижимости
7938,0,2194.220567,71,СРЕДНЕЕ,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
13864,0,2194.220567,72,среднее,1,женат / замужем,0,F,компаньон,0,NaN,сделка с автомобилем
10563,0,2194.220567,72,Среднее,1,Не женат / не замужем,4,F,пенсионер,1,NaN,приобретение автомобиля
18664,1,2194.220567,73,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,дополнительное образование


значения столбца total_income, это количественная переменная. тип пропуска полностью случайный.

In [ ]:
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

для каждого типа занятости сформируем медиану

In [ ]:
worker_median = data[data['income_type']=='сотрудник']['total_income'].median()

In [ ]:
partner_median = data[data['income_type']=='компаньон']['total_income'].median()

In [ ]:
pensioner_median = data[data['income_type']=='пенсионер']['total_income'].median()

In [ ]:
state_median= data[data['income_type']=='госслужащий']['total_income'].median()

In [ ]:
owner_median= data[data['income_type']=='предприниматель']['total_income'].median()

для каждого пропуска заполняем соответсвующее значение медианы

In [ ]:
data.loc[(data['income_type']=='сотрудник') & data['total_income'].isna(),'total_income'] = worker_median

In [ ]:
data.loc[(data['income_type']=='компаньон') & data['total_income'].isna(),'total_income'] = partner_median

In [ ]:
data.loc[(data['income_type']=='пенсионер') & data['total_income'].isna(),'total_income'] = pensioner_median

In [ ]:
data.loc[(data['income_type']=='госслужащий') & data['total_income'].isna(),'total_income'] = state_median

In [ ]:
data.loc[(data['income_type']=='предприниматель') & data['total_income'].isna(),'total_income'] = owner_median

проверим результат

In [ ]:
data['total_income'].isna().sum()

0

**Вывод**

* children: 
 * тип полностью случайный
 * количественная переменная
 * причина появления: опечатка при вводе
 * обработал заменой
* days_employed
 * тип полностью случайный
 * количественная переменная
 * причина появления: возможно это опечатка при вводе, отсутствие данных при вводе
 * обработал заменой на медиану
* dob_years
 * тип полностью случайный
 * количественная переменная
 * причина появления: опечатка при вводе
 * обработал заменой на среднее
* gender
 * тип полностью случайный
 * категориальная переменная
 * причина появления: возможно это опечатка при вводе
 * обработал заменой на часто встречающийся (можно было удалить)
* total_income
 * тип полностью случайный
 * количественная переменная
 * причина появления: отсутствие данных при вводе
 * разделил н группы по типу занятости и заменил на медиану для каждой группы

### Замена типа данных

перевожу значения total_income из float в int

In [ ]:
data['total_income']=data['total_income'].astype('int')

перевожу значения days_employed из float в int

In [ ]:
data['days_employed']=data['days_employed'].astype('int')

проверяю информацию/результат о типах данных

In [ ]:
data.dtypes

children             int64
days_employed        int32
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int32
purpose             object
dtype: object

In [ ]:
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья


**Вывод**

при переводе в тип int использовался метод astype() т.к. переводим в целое число из вещественного

### Обработка дубликатов

__education__

взглянем на значения столбца 

In [ ]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

переведем в нижний регистр education

In [ ]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

__family_status__

In [ ]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

перевожу в нижний регистр 

In [ ]:
print(data['family_status'].unique())
data['family_status'] = data['family_status'].str.lower()
print(data['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


__дубликаты строк__

взглянем на дубликаты

In [ ]:
data.loc[data.duplicated(keep=False)].sort_values(by='dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,2194,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
8853,1,2194,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
15892,0,2194,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,2194,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
3452,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9604,0,2194,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,2194,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы


поиск duplicated() и удаление drop_duplicates() полных добликатов строк 

In [ ]:
data.duplicated().sum()
data=data.drop_duplicates().reset_index(drop= True)

проверка обработки

In [ ]:
data.duplicated().sum()

0

**Вывод**

возможная причина полных дубликатов:
* заполнение файла из разных систем банка, в которых хранилась информация об одинаковых клиентах
* человечески фактор

### Лемматизация

для лемматизатора создал функцию. и прогнал по столбцу. добавил новый столбец с леммами

In [ ]:
#на вход передается значение из столбца purpose
#возвращает леммы
def purpose_lemm(purpose):
    return m.lemmatize(purpose)

In [ ]:
#data['purpose_lemmas'] = data['purpose'].apply(purpose_lemm)

In [ ]:
data['purpose_lemmas'] = data.loc[:100,'purpose'].apply(purpose_lemm)

Wall time: 0 ns


объеденим все леммы в одну строку. подсчитаем слова. выведим самые популярные

In [ ]:
lemmas_str=data['purpose_lemmas'].sum()

подсчет слов в списке целей получения кредита, используется коллекция Counter

In [ ]:
from collections import Counter
cnt_lemmas=Counter(lemmas_str)

In [ ]:
cnt_lemmas.most_common(10)

**Вывод**

Лидеры в целях получения кредита:
1. приобретение жилья/недвижимости
2. покупка автомобиля 
3. на образование
4. на свадьбу

### Категоризация данных

__классификация клиентов по наличию детей__

In [ ]:
#На вход функции попадает кол-во детей, а возвращает категорию клиента,
# 0 - детей нет
# 1 - дети есть
def child_group(child):
    if child == 0:
        return 0
    else:
        return 1       


добавляем новый столбец

In [ ]:
data['child_group_id'] = data['children'].apply(child_group)
data.head(5)

__создаем словарь статусов семейного положения__

In [ ]:
family_status_dict=data[['family_status','family_status_id']]

In [ ]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

In [ ]:
family_status_dict.sort_values('family_status_id')

__классифицируем клиентов по уровню дохода__

In [ ]:
data.groupby('total_income')['total_income'].count().head(3)

In [ ]:
data.groupby('total_income')['total_income'].count().tail(5)

In [ ]:
data.sort_values(by='total_income').head(5)

In [ ]:
# <= 52 000 -  low-income, ниже среднемесячной зарплаты по Ростату
# <= 100 000 - medium-income, по данным Росстата, у 4% жителей России доходы превышают 100 000 Р
# >100 000 - rich
def income_group(income):
    if income <= 52000:
        return 'low-income'
    elif income <= 100000:
        return 'medium-income'
    else:
        return 'rich'
        

добавляем новый столбец с группой дохода total_income_id

In [ ]:
data['total_income_id'] = data['total_income'].apply(income_group)
data.head(5)

In [ ]:
data.groupby('total_income_id')['debt'].count()

**группировка по целям получения кредита с подсчетом общего кол-ва и кол-ва задолжностей**

закомментировал прошлое решение, как не самое удачное

In [ ]:
#data_purpose = data.groupby('purpose').agg({'debt': ['count', 'sum']})
#data_purpose

проведем классификацию по популярным целям кредита. воспользуюсь функцией purpose_lemm из 2.4

ещё раз самые популярные цели

In [ ]:
cnt_lemmas.most_common(10)

In [ ]:
# на входе цель кредита. 
# возвращает группу цели 'жилье', 'автомобиль', 'образование', 'свадьба'
# иначе 'другое'
def purpose_to_rank(purpuse):
    purp_lemm = purpose_lemm(purpuse)
    if ('недвижимость' in purp_lemm) or ('жилье' in purp_lemm) or ('операция' in purp_lemm):
        return 'жилье'
    elif 'автомобиль' in purp_lemm:
        return 'автомобиль'
    elif 'образование' in purp_lemm:
        return 'образование'
    elif 'свадьба' in purp_lemm:
        return 'свадьба'
    else:
        return 'другое'

применим функцию к целям и создадим столбец с результатами

In [ ]:
data['purpose_rank']=data['purpose'].apply(purpose_to_rank)

In [ ]:
data['purpose_rank'].value_counts()

можно обратить внимание что других целей кредита нет

**Вывод**

1. проведена классификация клиентов по наличию детей, добавлен новый столбец child_group_id
2. создан словарь статусов семейного положения family_status_dict
3. проведена классификация клиентов по уровню дохода, добавлен новый столбец total_income_id
4. проведена классификация по целям получения кредита, добавлен новый столбец purpose_rank

результаты категоризации данных будут использоваться при формировании ответов на вопросы

## Шаг 3. Проанализируем данные и ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data.groupby('child_group_id').agg({'debt': ['count', 'sum']})

In [ ]:
all_debt_child_0 = data.groupby('child_group_id')['debt'].sum()[0]
count_child_0 = data.groupby('child_group_id')['debt'].count()[0]
print('Имели задолжность без детей: {:.2%}'.format(all_debt_child_0/count_child_0))

In [ ]:
all_debt_child_1 = data.groupby('child_group_id')['debt'].sum()[1]
count_child_1 = data.groupby('child_group_id')['debt'].count()[1]
print('Имели задолжность с детьми: {:.2%}'.format(all_debt_child_1/count_child_1))

**Вывод**

* Клиенты без детей чаще возвращают кредит в срок (92.5% без задолжности), чем клиенты с детьми (90.8% без задолжности).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

для расчета задолжности воспользуюсь методом mean для столбца debt и выведу результат в сводной таблице функцией pivot_table сгруппировав по family_status

In [ ]:
data.pivot_table(index=['family_status'], values='debt', aggfunc='mean').sort_values(by='debt')

**Вывод**

* Клиенты со статусом "не женат / не замужем" чаще имели задолжность по кредиту (9.75%).
* Клиенты со статусом "вдовец / вдова" реже имели задолжность по кредиту (6.57%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

кол-во клиентов имевших задолжность.
столбец total_income_id сформирован на этапе группировки данных

In [ ]:
data.groupby('total_income_id')['debt'].sum()

для расчета задолжности воспользуюсь методом mean для столбца debt и выведу результат в сводной таблице функцией pivot_table сгруппировав по уровню дохода total_income_id

In [ ]:
data.pivot_table(index=['total_income_id'], values='debt', aggfunc='mean').sort_values(by='debt')

**Вывод**

* Клиенты с уровнем дохода от 100 000 рублей в месяц чаще имели задолжность (8.1%) 
* Клиенты с уровнем дохода "ниже среднемесячного" по стране (до 52 000) реже имели задолжность (6.7%)

- Как разные цели кредита влияют на его возврат в срок?

для расчета задолжности воспользуюсь методом mean для столбца debt и выведу результат в сводной таблице функцией pivot_table сгруппировав по целям кредита purpose_rank, сформировнные при категоризации

In [ ]:
data.pivot_table(index=['purpose_rank'], values='debt', aggfunc='mean').sort_values(by='debt')

**Вывод**

* меньше всего задолжности при покупке жилья (7.2%).
* кол-во задолжности увеличивается при получении кредита на покупку автомобиля (9.4%) или образования(9.2%)

## Шаг 4. Общий вывод

1. вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?"
* Количество детей влияет на факт погашения кредита в срок:
 * Клиенты без детей имели задолжность по кредиту в 7.54% случаев.
 * Клиенты с детьми имели задолжность по кредиту в 9.21% случаев.
2. вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?"
* Семейное положение влияет на факт погашения кредита в срок:
 * Клиенты со статусом "не женат / не замужем" имели задолжность по кредиту в 9.8% случаев.
 * Клиенты со статусом "вдовец / вдова" имели задолжность по кредиту в 6.6% случаев.
3. вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"
* Уровень дохода влияет на факт погашения кредита в срок:
 * Клиенты с доходом "ниже среднемесячного" по стране имели задолжность по кредиту в 6.7% случаев.
 * Клиенты с доходом выше 100 000 рублей имели задолжность по кредиту в 8.2% случаев.
4. вопрос: "Как разные цели кредита влияют на его возврат в срок?"
* Цели кредита влияет на факт погашения в срок:
 * Клиенты с кредитом на жилье/недвижимость имели задолжность по кредиту в 7.2% случаев.
 * Клиенты с кредитом на автомобиль имели задолжность по кредиту в 9.4% случаев.
5. в столбцах days_employed и total_income около 10% значений имели пропущенные данные.
6. около 16% значений в days_employed (общий трудовой стаж в днях) были заполненны данными "невозвожными" для трудового стажа.  
